<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>Interactive Visualization</h1>
<h1>matplotlib animations</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D

from IPython import display

import watermark

%load_ext watermark
%matplotlib notebook
%matplotlib notebook

# Turn off matplotlib plot in Notebook
plt.ioff()

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 22.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 16
Architecture: 64bit

Git hash: bc90a514f321dab7787a0ed2da43bf0a6eccd05b

numpy     : 1.24.2
watermark : 2.1.0
json      : 2.0.9
IPython   : 7.19.0
pandas    : 1.5.3
matplotlib: 3.3.2



Let us start with the simplest example of an animation

In [3]:
fig = plt.figure()
ax = plt.axes(xlim=(0, 4), ylim=(-2, 2))
line, = ax.plot([], [], lw=3)

def init():
    line.set_data([], [])
    return line,

def animate(i):
    x = np.linspace(0, 4, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return line,

anim = FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True,
                               repeat=True)

# Display the animation as an HTML video
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)

Read the volcano data file

In [4]:
data = pd.read_csv('data/volcano.csv')

Transform to column format

In [5]:
df=data.unstack().reset_index()
df.columns=["X","Y","Z"]

Transform the old column name into a numeric value

In [6]:
df['X']=pd.Categorical(df['X'])
df['X']=df['X'].cat.codes

In [7]:
%%time

fig = plt.figure()
ax = fig.gca(projection='3d')

# Plot the surface of the vulcano
ax.plot_trisurf(df['Y'], df['X'], df['Z'], 
                cmap=plt.cm.viridis, linewidth=0.2)

def animate(angle):
    ax = plt.gca()
    
    # Rotate the figure by steps of 5 degrees
    ax.view_init(30, angle)

anim = FuncAnimation(fig, animate,
                        frames=200, interval=20, blit=True,
                        repeat=True)

# Display the animation as an HTML video
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)

CPU times: user 2min 35s, sys: 1min 1s, total: 3min 36s
Wall time: 1min 8s


Save GIF file

In [8]:
%%time

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)

def animate(angle):
    ax = plt.gca()
    ax.view_init(30, angle)

anim = FuncAnimation(fig, animate,
                        frames=200, interval=20, blit=True,
                        repeat=True)

# Save the animation as a GIF
anim.save('data/volcano.gif')

CPU times: user 2min 34s, sys: 1min 2s, total: 3min 37s
Wall time: 1min 9s


In [9]:
!open -a /Applications/Firefox.app data/volcano.gif

Save MP4 file

In [10]:
%%time

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)

def animate(angle):
    ax = plt.gca()
    ax.view_init(30, angle)

anim = FuncAnimation(fig, animate,
                        frames=200, interval=20, blit=True,
                        repeat=True)

writervideo = FFMpegWriter(fps=60) 
anim.save("data/volcano.mp4", writer=writervideo)

CPU times: user 2min 37s, sys: 1min 2s, total: 3min 39s
Wall time: 1min 8s


In [11]:
!open data/volcano.mp4

Saving MOV file

In [12]:
%%time

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_trisurf(df['Y'], df['X'], df['Z'], cmap=plt.cm.viridis, linewidth=0.2)

def animate(angle):
    ax = plt.gca()
    ax.view_init(30, angle)

anim = FuncAnimation(fig, animate,
                        frames=200, interval=20, blit=True,
                        repeat=True)

writervideo = FFMpegWriter(fps=60) 
anim.save("data/volcano.mov", writer=writervideo)

CPU times: user 2min 37s, sys: 1min 5s, total: 3min 42s
Wall time: 1min 14s


In [13]:
!open data/volcano.mov

<div style="width: 100%; overflow: hidden;">
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</div>